In [2]:
import pandas as pd
import numpy as np
import csv

In [3]:
ratings_list = []
movies_list = []
with open('./ratings.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        ratings_list.append(row)
with open('./movies.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',', quotechar='"')
    for row in csv_reader:
        movies_list.append(row)
ratings_list = ratings_list[1:10000] 
movies_list = movies_list[1:] 

In [4]:
#take some to score later
ratings_df_test = []
i = 1
while i < len(ratings_list):
    ratings_df_test.append(dict(userID = ratings_list[i][0], movieID = ratings_list[i][1], real = ratings_list[i][2]))
    ratings_list[i][2] = 0
    i += 3
len(ratings_df_test)

3333

In [5]:
ratings_df = pd.DataFrame(ratings_list, columns = ['userID', 'movieID', 'rating', 'timestamp'])
movies_df = pd.DataFrame(movies_list, columns = ['movieID', 'title', 'genres'])

In [6]:
R_df = ratings_df.pivot(index = 'userID', columns ='movieID', values = 'rating').convert_objects(convert_numeric=True).fillna(0)
R_df.head(10)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


movieID,1,10,100,100383,1004,100498,1005,1006,100714,1009,...,98961,991,99114,99149,994,99532,996,99728,99813,999
userID,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)
R_demeaned

array([[ 3.78868863, -0.21131137, -0.21131137, ..., -0.21131137,
        -0.21131137, -0.21131137],
       [-0.0964885 , -0.0964885 , -0.0964885 , ..., -0.0964885 ,
        -0.0964885 , -0.0964885 ],
       [-0.05003108,  2.94996892, -0.05003108, ..., -0.05003108,
        -0.05003108, -0.05003108],
       ...,
       [-0.10083903, -0.10083903, -0.10083903, ..., -0.10083903,
        -0.10083903, -0.10083903],
       [-0.03387197, -0.03387197, -0.03387197, ..., -0.03387197,
        -0.03387197, -0.03387197],
       [-0.03138595, -0.03138595, -0.03138595, ..., -0.03138595,
        -0.03138595, -0.03138595]])

In [14]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)
sigma = np.diag(sigma)
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

compare_rate = []
for test_rating in ratings_df_test:
    uid = test_rating['userID']
    mid = test_rating['movieID']
    real =  test_rating['real']
    pred = preds_df.get_value(int(test_rating['userID'])-1, test_rating['movieID'])
    compare_rate.append(dict(userID= uid, movieID= mid, real= real, predicted= pred))
compare_rate[:100]

[{'movieID': '3',
  'predicted': 0.0013227235022782446,
  'real': '4.0',
  'userID': '1'},
 {'movieID': '50',
  'predicted': -0.011454049877887701,
  'real': '5.0',
  'userID': '1'},
 {'movieID': '110',
  'predicted': 0.00803794983464956,
  'real': '4.0',
  'userID': '1'},
 {'movieID': '163',
  'predicted': 0.0023152646397238408,
  'real': '5.0',
  'userID': '1'},
 {'movieID': '231',
  'predicted': -0.016608353054806707,
  'real': '5.0',
  'userID': '1'},
 {'movieID': '296',
  'predicted': 0.046751478382445644,
  'real': '3.0',
  'userID': '1'},
 {'movieID': '349',
  'predicted': 0.025871580184758824,
  'real': '4.0',
  'userID': '1'},
 {'movieID': '367',
  'predicted': -0.04144315104162041,
  'real': '4.0',
  'userID': '1'},
 {'movieID': '457',
  'predicted': 0.031479956722019004,
  'real': '5.0',
  'userID': '1'},
 {'movieID': '527',
  'predicted': -0.013974320065950924,
  'real': '5.0',
  'userID': '1'},
 {'movieID': '553',
  'predicted': 0.02034793828277634,
  'real': '5.0',
  'use